<a href="https://colab.research.google.com/github/iupui-soic/pybootcamp/blob/main/Sanjana_Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from sklearn.datasets import fetch_openml

In [57]:
mnist=fetch_openml('mnist_784', as_frame=False)

In [58]:
X, y=mnist.data, mnist.target
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
X.shape

(70000, 784)

In [60]:
y.shape

(70000,)

In [61]:
import matplotlib.pyplot as plt
def plot_digit(image_data):
  image=image_data.reshape(28,28)
  plt.imshow(image,cmap='binary')
  plt.axis('off')
  some_digit=X[0]
  plot_digit(some_digit)
  plt.show()

In [62]:
y[0]

'5'

In [63]:
some_digit=X[0]
some_digit

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

In [64]:
X_train, X_test, y_train, y_test= X[:60000],X[60000:],y[:60000],y[60000:]

In [65]:
#training a binary classifier
y_train_5=(y_train=='5') #True for all 5's and becomes false for all other digits
y_test_5=(y_test=='5')

In [66]:
#Creating a Stochastic Gradient Descent (SGD) Classifier and training it on whole dataset
#SGD deals with training instances independently
from sklearn.linear_model import SGDClassifier
sgd_clf=SGDClassifier(random_state=42)
sgd_clf.fit(X_train,y_train_5)

SGDClassifier(random_state=42)

In [67]:
sgd_clf.predict([some_digit])

array([ True])

In [68]:
#Performance measure
#using k-fold cross validation (splitting the training set into k number of folds (here, 3))
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf,X_train,y_train_5,cv=3,scoring='accuracy')


array([0.95035, 0.96035, 0.9604 ])

In [69]:
#using a dummy classifier to classify every single image into the most frequent class
#In this case, is the negative class (non-5)
from sklearn.dummy import DummyClassifier
dummy_clf=DummyClassifier()
dummy_clf.fit(X_train,y_train_5)
print(any(dummy_clf.predict(X_train)))

False


In [70]:
#testing this model's accuracy
cross_val_score(dummy_clf, X_train,y_train_5,cv=3,scoring='accuracy')

array([0.90965, 0.90965, 0.90965])

In [71]:
#over 90% accuracy is seen because only 10% of images are 5s, so if we always guess that an image is not a 5, we are right 90% of time
#this shows why accuracy is not a preferred performamce measure for classifiers (especially in skewed datasets)

In [72]:
#evaluating performance using confusion matrix
#to compute the confusion matrix, we need to have a set of predictions so that they can be compared to the actual targets
from sklearn.model_selection import cross_val_predict
y_train_pred=cross_val_predict(sgd_clf,X_train,y_train_5,cv=3)
#returns predictions made on each fold

In [73]:
#using confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_train_5,y_train_pred)
cm
#each row represents actual class
#each column represents predicted class
#53892 were correctly classified as non-5's (TN), 687(FP)
#second row: 1891 (FN) and 3530 (TP)

array([[53892,   687],
       [ 1891,  3530]])

In [74]:
#computing a confusion matrix for a perfect classifier which only contains TP,TN
y_train_perfect_predictions=y_train_5
confusion_matrix(y_train_5,y_train_perfect_predictions)

array([[54579,     0],
       [    0,  5421]])

In [75]:
#precision =TP/TP+FP
#Recall =TP/TP+FN (sensitivity or TPR)
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_5,y_train_pred)#==3530/(687+3530)

0.8370879772350012

In [76]:
recall_score(y_train_5,y_train_pred)#==3530/(1891+3530)

0.6511713705958311

In [77]:
#now we see that when the classifier claims an image represents as 5, it is right only 83.7% of time
#Moreover it only detects 65.1% of all the 5's
#combining precision and recall into a single metric i.e., F1 score
#F1 is the harmonic mean of precision and recall
#harmonic mean is different from regular mean, the former gives weight to low values
#As a result, the classifier will only get a high F1 score if both precision and recall are high
from sklearn.metrics import f1_score
f1_score(y_train_5,y_train_pred)

0.7325171197343846

In [83]:
#defining y_scores
y_scores=sgd_clf.decision_function([some_digit])
y_scores


array([2164.22030239])